In [ ]:
%%time

import itertools
import os 
import sys
import re
import math
import random
import requests
import seaborn as sns
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
from tensorflow.keras import layers
import matplotlib.pylab as plt
import numpy as np
import PIL.Image as Image
import pandas as pd
import tensorflow as tf
import keras
from tensorflow.keras.layers import Input
from keras.models import Sequential, Model
from keras.layers import LSTM, BatchNormalization, Flatten, Input, Dense,Dropout,Activation,Flatten,Reshape,Lambda,Embedding,Conv2D,MaxPooling2D, UpSampling2D   #卷积层，池化层
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow.python.platform.build_info as build 
import tensorflow_hub as hub 
import tensorflow_datasets as tfds 
from tensorflow.python.client import device_lib

%cd C:\...

################################################################################

def getTemp(starting_frame, ending_frame, info_current_dataset, img_current_dataset):
  for frame in range(starting_frame, ending_frame + 1):
    temp_filename = "TEMPERATURE_at_frame%d.txt" % frame 
    time_stamp_raw = pd.read_csv(temp_filename, sep='\t', header=None, 
                                skiprows=[i for i in range(0,4)], on_bad_lines='skip')
    timestamp = [float(s) for s in re.findall(r':\s(.*)', time_stamp_raw.iat[2, 1])]
    timestamp = timestamp[0]    
      
    temp_raw = pd.read_csv(temp_filename, header=None, sep='\t', 
                          skiprows=[i for i in range(0,9)]).iloc[[i for i in range(0, 17583 - 9)], [0, 1]]
    temp_raw.columns = ['node_label', 'temperature']
    temp = pd.merge(temp_raw, node_info, on = 'node_label')
    temp['step'] = frame
    temp['temperature'] = temp['temperature'] - 273.15

    if (frame == starting_frame):
      initial_time = timestamp
      x_dim = (temp['node_x'].unique()).size
      y_dim = (temp['node_y'].unique()).size
      z_dim = (temp['node_z'].unique()).size

    temp['time'] = (timestamp - initial_time)/60   
    temp['scaled_x'] = ( temp['node_x'] / max(temp['node_x']) ) * x_dim
    temp['scaled_y'] = ( (temp['node_y']) / (max(temp['node_y'])) ) * y_dim
    temp['scaled_z'] = ( (temp['node_z']) / (max(temp['node_z'])) ) * z_dim

    temp['X'] = temp.scaled_x.astype('int')
    temp['Y'] = temp.scaled_y.astype('int')
    temp['Z'] = temp.scaled_z.astype('int')
    info_current_dataset.append(temp)                     # append current frame
    
    img_x = max(temp['X']) + 1   # 102
    img_y = max(temp['Y']) + 1   # 30 
    img_z = max(temp['Z']) + 1   # 6  

    get_image = np.zeros(shape = (img_y, img_x, img_z))

    for i in range(len(temp)):
      x_coordinate = temp.loc[i, "X"]
      y_coordinate = temp.loc[i, "Y"]
      z_coordinate = temp.loc[i, "Z"]
      get_image[y_coordinate][x_coordinate][z_coordinate] = temp.loc[i, "temperature"]
    img_current_dataset.append(get_image)                 # append current frame
 
def getImgChannel(channel, dataset):
  output_img = []
  length = len(dataset)
  for i in range(length):
    output_img.append(dataset[i, :, :, channel])
  return np.array(output_img)

def get_all_ImgChannel(dataset):
  output_img = []
  length = len(dataset)
  for i in range(length):
    output_img.append(dataset[i, :, :, 0])
  for i in range(length):
    output_img.append(dataset[i, :, :, 1])
  for i in range(length):
    output_img.append(dataset[i, :, :, 2])
  for i in range(length):
    output_img.append(dataset[i, :, :, 3])
  for i in range(length):
    output_img.append(dataset[i, :, :, 4])
  for i in range(length):
    output_img.append(dataset[i, :, :, 6])
  return np.array(output_img)

def patch_image(dataset):
  for img in range(len(dataset)):
    for col in range(102):
      (dataset[img])[28, col] = ((dataset[img])[27, col] + (dataset[img])[29, col]) / 2
    for row in range(30):
      (dataset[img])[row, 100] = ((dataset[img])[row, 99] + (dataset[img])[row, 101]) / 2
    (dataset[img])[28, 100] = ((dataset[img])[27, 99] + (dataset[img])[27, 101] + (dataset[img])[29, 99] + (dataset[img])[29, 101]) / 4

################################################################################
node_info = pd.read_csv('Mesh_info.txt', delim_whitespace=True, header=None)
node_info.columns = ['node_label', 'node_x', 'node_y', 'node_z']
node_info = node_info.astype(str).astype(float)

temperature_info_all = [] 
temperature_img_all = []

getTemp(1, 541, temperature_info_all, temperature_img_all)

temperature_img_all_array = np.array(temperature_img_all)

layer_z6 = getImgChannel(6, temperature_img_all_array)
patch_image(layer_z6)
layer_z6_norm = layer_z6 / np.max(layer_z6)
max_temp = np.max(layer_z6)

images_all_layers = get_all_ImgChannel(temperature_img_all_array)
patch_image(images_all_layers)
images_all_layers_norm = images_all_layers / np.max(images_all_layers)

In [ ]:
%%time

train = []

# split into test and training datasets
test_size = 0.2
ratio = int((1 - test_size) * layer_z6_norm.shape[0])
training_dataset = layer_z6_norm[:ratio]
testing_dataset = layer_z6_norm[ratio:]

ratio_sub = int((1 - test_size) * training_dataset.shape[0])
train1 = training_dataset[:ratio_sub]
test1 = training_dataset[ratio_sub:]

train = train1.reshape(train1.shape[0], train1.shape[1] * train1.shape[2])
validation = test1.reshape(test1.shape[0], test1.shape[1] * test1.shape[2])
test = testing_dataset.reshape(testing_dataset.shape[0], testing_dataset.shape[1] * testing_dataset.shape[2])

train_X, validation_X, test_X = train, validation, test;
train_X = train_X.reshape(train_X.shape[0], 30, 102, 1);
validation_X = validation_X.reshape(validation_X.shape[0], 30, 102, 1);
test_X = test_X.reshape(test_X.shape[0], 30, 102, 1);

#################################################################################################################################
# GET TRAINED CNN autoencoder

autoencoder = tf.keras.models.load_model('C:/Users/trained_CNN')
encoder = Model(inputs = autoencoder.input, outputs = autoencoder.layers[8].output)

deep_X_train = encoder.predict(train_X)
deep_X_validation = encoder.predict(validation_X)
deep_X_test = encoder.predict(test_X)

deep_scalar = MinMaxScaler().fit(deep_X_train)
deep_X_train = np.array(pd.DataFrame(deep_scalar.transform(deep_X_train)))
deep_X_validation = np.array(pd.DataFrame(deep_scalar.transform(deep_X_validation)))
deep_X_test = np.array(pd.DataFrame(deep_scalar.transform(deep_X_test)))

def get_Time_Step_Data(time_step, data):
  dataArray = []
  for i in range(len(data) - time_step):  
    tempArray = []
    for j in range(time_step):
      tempArray.append(data[i + j])
    dataArray.append(tempArray)
  return np.array(dataArray)

TIME_STEP = 5

RNN_Y_train = deep_X_train[TIME_STEP:]
RNN_X_train = get_Time_Step_Data(TIME_STEP, deep_X_train)
RNN_Y_validation = deep_X_validation[TIME_STEP:]
RNN_X_validation = get_Time_Step_Data(TIME_STEP, deep_X_validation)
RNN_Y_test = deep_X_test[TIME_STEP:]
RNN_X_test = get_Time_Step_Data(TIME_STEP, deep_X_test)

In [ ]:
%%time

#**************************** RNN MODEL training ************************
rnn_hidden_units = 15
lr = 0.001
epoch = 1000
batch_size = 12

############ 128 DEEP FEATURES, RNN SETUP #####################

opt = keras.optimizers.Adam(learning_rate = lr, beta_1 = 0.9, beta_2 = 0.999, 
                            epsilon = None, decay = 0.000, amsgrad = False)
RNN_model = keras.Sequential()

RNN_model.add(layers.SimpleRNN(rnn_hidden_units, activation = 'softsign' ,
                    input_shape = (RNN_X_train.shape[1], RNN_X_train.shape[2]),
                    return_sequences = 1))
RNN_model.add(layers.SimpleRNN(30, activation = 'tanh', return_sequences=0))
RNN_model.add(layers.Dense(128))

callbacks = [EarlyStopping(monitor = 'val_loss', patience = 100, verbose=2),
             ModelCheckpoint('RNN777.h5', monitor = 'val_loss', save_best_only = True, verbose = 0)]
keras.backend.set_epsilon(1)
RNN_model.compile(loss = 'mae', optimizer = opt, metrics=['mape'])
RNN_model_history = RNN_model.fit(RNN_X_train, RNN_Y_train, epochs = epoch, callbacks = callbacks,
                        batch_size = batch_size, verbose = 1, shuffle = True,
                        validation_data = (RNN_X_validation, RNN_Y_validation))

plt.plot(RNN_model_history.history['mape'], label='Training data')
plt.plot(RNN_model_history.history['val_mape'], label ='Validation data')
plt.ylabel('MAPE')
plt.xlabel('No. epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
RNN_model_testing = RNN_model.evaluate(RNN_X_test, RNN_Y_test, verbose = 2)
RNN_model_testing

In [ ]:
%%time

#####################################################################################
################ MAKE PREDICTIONS BASED ON TRAINED MODEL ############################
#### Load the trained LSTM and RNN models

RNN_file = 'C:/trained_RNN'
RNN_model = tf.keras.models.load_model(RNN_file)
LSTM_file = 'C:/trained_LSTM'
LSTM_model = tf.keras.models.load_model(LSTM_file)
########################  Decoder ########################
features = 128 
deep_pred = Input(shape=(features, ))
x = Reshape((1, 1, features), name = 'D1')(deep_pred)
x = UpSampling2D((2, 2), name = 'D2')(x)
x = Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'D3')(x)
x = UpSampling2D((2, 2), name = 'D4')(x)
x = Conv2D(32, (3, 3), activation = 'relu', padding = 'same', name = 'D5')(x)
x = UpSampling2D((3, 3), name = 'D6')(x)
x = Conv2D(16, (3, 3), activation = 'relu', name = 'D7')(x)
x = UpSampling2D((3, 3), name = 'D8')(x)
x = Conv2D(16, (1, 3), activation = 'relu', name = 'D9')(x)
x = Conv2D(16, (1, 3), activation = 'relu', name = 'D10')(x)
x = UpSampling2D((1, 4), name = 'D11')(x)
decoded = Conv2D(1, (1, 3), activation = 'relu', name = 'D12')(x)
decoder = Model(deep_pred, decoded)
# decoder.summary()
decoder.layers[3].set_weights([autoencoder.layers[11].get_weights()[0], 
                               autoencoder.layers[11].get_weights()[1]])
decoder.layers[5].set_weights([autoencoder.layers[13].get_weights()[0],
                               autoencoder.layers[13].get_weights()[1]])
decoder.layers[7].set_weights([autoencoder.layers[15].get_weights()[0],
                               autoencoder.layers[15].get_weights()[1]])
decoder.layers[9].set_weights([autoencoder.layers[17].get_weights()[0],
                               autoencoder.layers[17].get_weights()[1]])
decoder.layers[10].set_weights([autoencoder.layers[18].get_weights()[0],
                                autoencoder.layers[18].get_weights()[1]])
decoder.layers[12].set_weights([autoencoder.layers[20].get_weights()[0],
                                autoencoder.layers[20].get_weights()[1]])

predict_RNN_all = []
predict_LSTM_all = []

for i in range(6):
    Deep_temporal_current_layer = Deep_temporal_X_all[i]
    max_temp = max_temp_all[i]
    
    LSTM_predict_hat = LSTM_model.predict(Deep_temporal_current_layer, batch_size = 32)    
    RNN_predict_hat = RNN_model.predict(Deep_temporal_current_layer, batch_size = 32)    
    
    def deep_i2r(yp):
        yr = deep_scalar.inverse_transform(yp.reshape(-1, features)) # 128 features
        return yr
    LSTM_deep_pred = deep_i2r(LSTM_predict_hat)
    RNN_deep_pred = deep_i2r(RNN_predict_hat)

    Pred_norm_LSTM = decoder.predict(LSTM_deep_pred)
    Pred_norm_RNN = decoder.predict(RNN_deep_pred)

    Pred_LSTM = max_temp * (Pred_norm_LSTM.reshape(Pred_norm_LSTM.shape[0], 
                                                         Pred_norm_LSTM.shape[1], 
                                                         Pred_norm_LSTM.shape[2]))
    Pred_RNN = max_temp * (Pred_norm_RNN.reshape(Pred_norm_RNN.shape[0], 
                                                       Pred_norm_RNN.shape[1], 
                                                       Pred_norm_RNN.shape[2]))
    predict_LSTM_all.append(Pred_LSTM)
    predict_RNN_all.append(Pred_RNN)
    
predict_LSTM_all = np.array(predict_LSTM_all)
predict_RNN_all = np.array(predict_RNN_all)

In [ ]:
%%time

def calculate_percent_MAPE_at_each_point(ori_data, pred_data):
  row = ori_data.shape[0]
  col = ori_data.shape[1]
  mae = np.zeros(shape = (row, col))
  for y in range(row):
    for x in range(col):
      ori_val = ori_data[y][x]
      pred_val = pred_data[y][x]
      mae[y][x] = np.absolute(ori_val - pred_val) / ori_val * 100
  return mae

############################## MAIN FUNCTION #############################
x_time = []
x_step = []
accuracy_LSTM = []
accuracy_RNN = []
accuracy_2D_calculate_each_point_LSTM_all = []
accuracy_1D_avg_calculate_each_point_LSTM_all = []
accuracy_2D_calculate_each_point_RNN_all = []
accuracy_1D_avg_calculate_each_point_RNN_all = []
TIME_STEP = 5

for which_layer in range(6):
    layer_z6 = layer_img_all[which_layer]
    Prediction_lstm = predict_LSTM_all[which_layer]
    Prediction_rnn = predict_RNN_all[which_layer]
    
    accuracy_2D_calculate_each_point_LSTM = []
    accuracy_1D_avg_calculate_each_point_LSTM = []
    accuracy_2D_calculate_each_point_RNN = []
    accuracy_1D_avg_calculate_each_point_RNN = []
    
    end_frame = 536
    for step in range(0, end_frame, 1):
      time = (temperature_info_all[TIME_STEP + step])['time'].iloc[0]
      ori = layer_z6[TIME_STEP + step]
      the_pred_LSTM = Prediction_lstm[step]
      the_pred_RNN = Prediction_rnn[step]
    
      # Ploting
      #plot_images_comparison(ori, the_pred, time)
      the_pred_MAE_LSTM = the_pred_LSTM.reshape(the_pred_LSTM.shape[0] * the_pred_LSTM.shape[1])
      the_pred_MAE_RNN = the_pred_RNN.reshape(the_pred_RNN.shape[0] * the_pred_RNN.shape[1])
      ori_cal_MAE_ = ori.reshape(ori.shape[0] * ori.shape[1])
      
      if(which_layer == 5):
        x_time.append(time)
        x_step.append(step)

      # Calculate percentage MAE at each point in each frame
      the_result_LSTM = calculate_percent_MAPE_at_each_point(ori, the_pred_LSTM)
      accuracy_2D_calculate_each_point_LSTM.append(the_result_LSTM)
      accuracy_1D_avg_calculate_each_point_LSTM.append(np.average(the_result_LSTM))

      the_result_RNN = calculate_percent_MAPE_at_each_point(ori, the_pred_RNN)
      accuracy_2D_calculate_each_point_RNN.append(the_result_RNN)
      accuracy_1D_avg_calculate_each_point_RNN.append(np.average(the_result_RNN))

    accuracy_2D_calculate_each_point_LSTM_all.append(accuracy_2D_calculate_each_point_LSTM)
    accuracy_1D_avg_calculate_each_point_LSTM_all.append(accuracy_1D_avg_calculate_each_point_LSTM)
    accuracy_2D_calculate_each_point_RNN_all.append(accuracy_2D_calculate_each_point_RNN)
    accuracy_1D_avg_calculate_each_point_RNN_all.append(accuracy_1D_avg_calculate_each_point_RNN)

accuracy_1D_avg_calculate_each_point_LSTM_all = np.array(accuracy_1D_avg_calculate_each_point_LSTM_all)
accuracy_1D_avg_calculate_each_point_RNN_all = np.array(accuracy_1D_avg_calculate_each_point_RNN_all)
accuracy_2D_calculate_each_point_LSTM_all = np.array(accuracy_2D_calculate_each_point_LSTM_all)
accuracy_2D_calculate_each_point_RNN_all = np.array(accuracy_2D_calculate_each_point_RNN_all)

In [ ]:
%%time  
##### PLOT CONTOUR COLORMAP TO COMPARE PREDICTION RESULTS WITH THE ORIGINAL IMAGES #####
def plot_images_comparison(original, predicted, mape, mape_value, atTime, which_layer):
  fig, (ax1, ax2, ax3) = plt.subplots(nrows = 3, ncols=1, figsize=(8, 9), sharex = True)
  plt.tight_layout(pad = 1)
  ax1.set_title('Layer ' + str(which_layer+1) + ' at ' + str(round(atTime, 0)) + ' mins: ' + "\n" + "\n" + 
                     'True temperature (°C)', pad=11, fontsize = font_size)
  im1 = ax1.imshow(original, cmap='jet', aspect='auto')

  ax2.set_title('CNN-RNN predicted temperature (°C)', pad = 11, fontsize = font_size)
  im2 = ax2.imshow(predicted, cmap='jet', aspect='auto')

  ax3.set_title('MAPE (%) mapping with average ' + str(round(mape_value, 1)) + ' %', pad=11, fontsize = font_size)
  im3 = ax3.imshow(mape, cmap='jet', aspect='auto')
  
  #plt.setp(ax[0], xlabel='Length of composite (pixels)')
  plt.setp(ax3, xlabel='Length of composite (pixels)')
  plt.setp(ax2, ylabel='Width of composite (pixels)')

  fig.subplots_adjust(right = 0.86)
  cbar_ax1 = fig.add_axes([0.89, 0.48, 0.01, 0.3])
  cbar1_ticks = np.around(np.linspace(np.around(np.min(predicted), decimals = 1), 
                            np.around(np.max(predicted), decimals = 1), 6, endpoint=True), decimals = 1)
  cbar1 = fig.colorbar(im2, cax = cbar_ax1)
  cbar1.set_ticks(cbar1_ticks)
  cbar1.set_ticklabels((np.around(cbar1_ticks, decimals = 0)).astype(int))
  ax1.text(1.04, 0.55, '(°C)', transform=ax1.transAxes, fontsize = font_size, verticalalignment='top')

  cbar_ax2 = fig.add_axes([0.89, 0.1, 0.01, 0.15])
  cbar2_ticks = np.around(np.linspace(np.around(np.min(mape), decimals = 1), 
                            np.around(np.max(mape), decimals = 1), 4, endpoint=True), decimals = 1)
  cbar2 = fig.colorbar(im3, cax = cbar_ax2)
  cbar2.set_ticks(cbar2_ticks)
  cbar2.set_ticklabels(cbar2_ticks)
  ax2.text(1.04, -0.35, '(%)', transform=ax2.transAxes, fontsize = font_size, verticalalignment='top')

  font = {'family': 'Times New Roman', 'weight': 'normal', 'size': font_size}
  plt.rc('font', **font)
##########################################################################
############################# DRIVER PROGRAM RNN PREDICTION RESULTS#############################
which_layer = 2
whichStep = 150
font_size = 24
this_layer = layer_img_all[which_layer]

ori_IMG = this_layer[TIME_STEP + whichStep]
pred_IMG = predict_RNN_all[which_layer, whichStep]
mape_IMG = accuracy_2D_calculate_each_point_RNN_all[which_layer, whichStep]
mape_at_frame = accuracy_1D_avg_calculate_each_point_RNN_all[which_layer, whichStep]
time_ = ((temperature_info_all[TIME_STEP + whichStep])['time'].iloc[0]).astype('int')
plot_images_comparison(ori_IMG, pred_IMG, mape_IMG, mape_at_frame, time_, which_layer)